In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import requests
import concurrent.futures
import time

# Dataset Preparation for Sentence Transformers

## Part 1: Using raw CV text without making any updates or changes with language models

### Candidate Data

In [2]:
# Reading the candidate data and looking at its shape

df_candidate = pd.read_csv("DatasetwithExperiences.csv")
df_candidate.shape

(95817, 11)

In [3]:
# Looking at the column names
df_candidate.columns

Index(['Id_candidate', 'name', 'fileName', 'nationality', 'permis', 'address',
       'experiences', 'text', 'gender', 'postcode', 'jobTitle'],
      dtype='object')

In [4]:
# Are there any duplicate entries? are there any duplicates in the id_candidate column?

df_candidate['Id_candidate'].is_unique

False

In [5]:
# there are duplicate candidates. Lets's get rid of them. 
df_candidate.drop_duplicates(subset=['Id_candidate'], keep='first', inplace=True)


In [6]:
# Shape of the candidate data after dropping the duplicates

df_candidate.shape

(78723, 11)

In [7]:
df_candidate['text'].isnull().sum()

307

### Vacancies Data

In [8]:
# Reading the vacancies dataset and looking at the shape

df_vac = pd.read_csv('vacancies.csv', sep=';')
df_vac.shape

(13204, 16)

In [9]:
df_vac.head(3)

,Id,Titre,Région,Département,Ville,Fonction,Date d'activation,Type de poste,Type de formation,Salaire min (euro brut annuel),Salaire max (euro brut annuel),Profil attendu,Expérience,Descriptif de l'offre,Actif,Archivé
0,4,Candidature spontanée,NaN,NaN,NaN,NaN,20/06/2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oui,Non
1,13983,Commercial B to B itinérant (H/F) - Montbéliard,Franche Comté,25,Montbéliard,Commerce,17/04/2024,CDI,BAC+2/BTS,27000.0,32000.0,"Issu-e d'une formation commerciale, vous dispo...",Expérimenté/Pas de management,Commercial BtoB itinérant h/f - CDI Au sein d...,Oui,Non
2,13982,Responsable d'Agence (H/F) - Vitrolles,Provence Alpes Côte d'Azur,13,Vitrolles,Commerce,16/04/2024,CDI,BAC+2/BTS,24000.0,35000.0,"Vous avez une première expérience réussie, not...",Manager/Gestion d'équipe,Responsable d'Agence h/f - CDI Grâce à votre ...,Oui,Non


In [10]:
# getting rid of the first entry
df_vac = df_vac.iloc[1:].reset_index(drop=True)
df_vac.shape


(13203, 16)

In [11]:
# are there any duplicates in the id_vacancy?

df_vac['Id'].is_unique

# no duplicates

True

In [12]:
# Since we are preparing data for a sentence transformer that accepts sentence pairs as inputs, we need to combine all the information about the vacancy into a single unified piece of text which will be treated as a "sentence". 

df_vac_textual = pd.DataFrame(columns = ['Id_vacancy', 'vacancy_text'])

df_vac_textual['Id_vacancy'] = df_vac['Id'].unique()

df_vac_textual.shape

(13203, 2)

In [13]:
for i in range(df_vac_textual.shape[0]):
    details = ""
    details += "Titre: " + str(df_vac['Titre'][i]) if pd.notnull(df_vac['Titre'][i]) else "Titre: " + "" 
    details += "\nFonction: " + str(df_vac['Fonction'][i]) if pd.notnull(df_vac['Fonction'][i]) else "\nFonction: " + ""
    details += "\nVille et Region: " + (str(df_vac['Ville'][i]) if pd.notnull(df_vac['Ville'][i]) else "") + ", " + (str(df_vac['Région'][i]) if pd.notnull(df_vac['Région'][i]) else "")
    details += "\nType de poste: " + str(df_vac['Type de poste'][i]) if pd.notnull(df_vac['Type de poste'][i]) else "\nType de poste: " + ""
    details += "\nDescriptif de l'offre: " + str(df_vac["Descriptif de l'offre"][i]) if pd.notnull(df_vac["Descriptif de l'offre"][i]) else "\nDescriptif de l'offre: "
    details += "\nType de formation requise: " + str(df_vac['Type de formation'][i]) if pd.notnull(df_vac['Type de formation'][i]) else "\nType de formation requise: "
    details += "\nProfil attendu: " + str(df_vac['Profil attendu'][i]) if pd.notnull(df_vac['Profil attendu'][i]) else "\nProfil attendu: "

    df_vac_textual.loc[i, 'vacancy_text'] = details
    

In [14]:
df_vac_textual.shape

(13203, 2)

In [15]:
df_vac_textual.isnull().sum()

Id_vacancy      0
vacancy_text    0
dtype: int64

In [16]:
#df_vac_textual.to_csv('vacancies_textual.csv')

### Applications Data

In [17]:
# Reading the two application csv files and viewing their shape

df_app_pos = pd.read_csv("applications_positive.csv", sep=';')
df_app_neg = pd.read_csv("applications_negative.csv", sep=';')

df_app_pos.shape, df_app_neg.shape

C:\Users\Sinjini\AppData\Local\Temp\ipykernel_744\383347155.py:4: DtypeWarning: Columns (7,9,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_app_neg = pd.read_csv("applications_negative.csv", sep=';')


((34917, 15), (65716, 15))

In [18]:
# column names of both the datasets

print(df_app_pos.columns)
print(df_app_neg.columns)

# same column names

Index(['Id_application', 'Date', 'Id_candidate', 'Id_vacancy', 'Civilité',
       'Ville', 'status', 'Motif de refus', 'Métier', 'Tranche d'age',
       'Type de contrat', 'Prétentions salariales mensuelles fixe net',
       'Permis de conduire', 'Mobilité', 'Code postal'],
      dtype='object')
Index(['Id_application', 'Date', 'Id_candidate', 'Id_vacancy', 'Civilité',
       'Ville', 'status', 'Motif de refus', 'Métier', 'Tranche d'age',
       'Type de contrat', 'Prétentions salariales mensuelles fixe net',
       'Permis de conduire', 'Mobilité', 'Code postal'],
      dtype='object')


In [19]:
# 'status' values for both the datasets

print(df_app_pos['status'].value_counts())
print("-------------------------------------------------------------")
print(df_app_neg['status'].value_counts())

status
Poste pourvu    32657
Embauche         2260
Name: count, dtype: int64
-------------------------------------------------------------
status
Négatif + 2jrs       43123
Négatif              12476
Négatif + 5jrs        2606
Neg sans msg          2191
NET+ 3jrs             1035
NET                    848
NET Savoir Faire       759
NET Localisation       624
NET Rému               610
NAE                    435
NAE +5jrs              299
No news                242
NET Savoir Etre        171
NET Cond. Travail      134
Vivier                  97
NAE Vivier              52
Vivier + 2jrs           14
Name: count, dtype: int64


In [20]:
# Merging "app_pos" and "app_neg" to form a single applications dataset

df_app = pd.concat([df_app_pos, df_app_neg], ignore_index=True)
df_app.shape

(100633, 15)

In [21]:
df_app.isnull().sum()

Id_application                                     0
Date                                               0
Id_candidate                                       0
Id_vacancy                                         0
Civilité                                       15976
Ville                                           9284
status                                             0
Motif de refus                                 99551
Métier                                          1161
Tranche d'age                                  99396
Type de contrat                               100632
Prétentions salariales mensuelles fixe net     73782
Permis de conduire                             68965
Mobilité                                      100442
Code postal                                    13357
dtype: int64

In [22]:
df_app['Mobilité'].value_counts()

Mobilité
Ile-de-france                                          63
Haute-Normandie                                        19
Pays de la Loire                                       19
Internationale , Nationale                             18
Nord pas de Calais                                     10
Nationale                                               7
Rhônes Alpes                                            7
Bourgogne                                               5
Internationale , Rhônes Alpes                           5
Basse Normandie                                         4
Champagne Ardenne                                       4
Picardie                                                3
Midi Pyrénées                                           3
Bretagne                                                3
Internationale                                          3
Aquitaine                                               3
Aquitaine , Poitou Charentes                            2
Centr


The Mobilite column has many missing values. However, it is an important feature as it contains the 
candidate’s desired job location. The values that are not missing are regions, not cities. 
The Ville column (candidate’scurrent city) has negligible missing values. Hence, a new column 'Region'
 should be created by mapping each Ville to its corresponding region using the
 Wikidata API. The missing Mobilite values could then be filled with this new
 Region column. This means that if the candidate has specifically mentioned
 their wish of job location, we would take that into account, otherwise, we
 will take into account the region where the candidate currently lives.




In [23]:
# Creating a new region column first and initializing it with None. 

df_app['Region'] = None

In [24]:
def get_entity_data(qid):
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()['entities'][qid]

def get_instance_of_ids(entity):
    claims = entity.get('claims', {})
    if 'P31' not in claims:
        return []
    return [claim['mainsnak']['datavalue']['value']['id'] for claim in claims['P31']]

def find_region(qid, visited=None):
    if visited is None:
        visited = set()
    if qid in visited:
        return None
    visited.add(qid)

    entity = get_entity_data(qid)
    instance_of = get_instance_of_ids(entity)

    # If this entity is a French region (Q36784), return it
    if "Q36784" in instance_of:
        return entity['labels']['fr']['value']

    # Otherwise, go up the P131 chain
    p131_claims = entity.get('claims', {}).get('P131')
    if not p131_claims:
        return None

    for claim in p131_claims:
        parent_qid = claim['mainsnak']['datavalue']['value']['id']
        region = find_region(parent_qid, visited)
        if region:
            return region

    return None

def get_city_region(city_name):
    # Step 1: Search for the city
    search_url = "https://www.wikidata.org/w/api.php"
    search_params = {
        "action": "wbsearchentities",
        "search": city_name,
        "language": "en",
        "format": "json",
        "limit": 1
    }
    search_response = requests.get(search_url, params=search_params).json()
    if not search_response['search']:
        return f"No results found for {city_name}"
    entity_id = search_response['search'][0]['id']

    # Step 2: Walk the hierarchy to find region
    region = find_region(entity_id)
    return region or f"No region found for {city_name}"

In [25]:
# Step 1: Deduplicate cities
unique_cities = df_app['Ville'].dropna().unique()

# Step 2: Define thread-safe wrapper
def fetch_region(city):
    try:
        region = get_city_region(city)
        print(f"{city}: {region}")
        return (city, region)
    except Exception as e:
        print(f"Exception for {city}: {e}")
        return (city, None)

# Step 3: Fetch in parallel
city_region_map = {}
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    results = executor.map(fetch_region, unique_cities)
    for city, region in results:
        city_region_map[city] = region

print(f"\nFetched regions for {len(city_region_map)} unique cities in {round(time.time() - start_time, 2)} seconds.")


# Step 4: Apply the map back to DataFrame (only where Region is null)
df_app['Region'] = df_app.apply(
    lambda row: city_region_map.get(row['Ville']) if pd.isnull(row['Region']) and pd.notnull(row['Ville']) else row['Region'],
    axis=1
)


ST OUEN EN BELIN: No results found for ST OUEN EN BELIN
Le Mesle: No results found for Le Mesle
Alençon: Normandie
BOISSY SAINT LEGER: Île-de-France
salon de provence: Provence-Alpes-Côte d'Azur
Plouhinec: Bretagne
AUZOUER EN TOURAINE: Centre-Val de Loire
orléans: Centre-Val de Loire
NARBONNE: Occitanie
Marseille: Provence-Alpes-Côte d'Azur
croix: Hauts-de-France
REIMS: Grand Est
Lanildut: Bretagne
Halluin : Hauts-de-France
SAINT-Fons: Auvergne-Rhône-Alpes
belin-beliet: Nouvelle-Aquitaine
Bondy: Île-de-France
WATTRELOS: Hauts-de-France
MONDEVILLE: Normandie
CHALONS-EN-CHAMPAGNE: Grand Est
saint cloud: Île-de-France
Leuville sur orge: Île-de-France
Argenteuil: Île-de-France
Pessac: Nouvelle-Aquitaine
Vallangoujard: Île-de-France
RUEIL MALMAISON: Île-de-France
Paris 19e: Île-de-France
BOIS-GUILLAUME: Normandie
ATTICHY: Hauts-de-France
BOBIGNY: Île-de-France
Choisy-Le-Roi: Île-de-France
Velaux: Provence-Alpes-Côte d'Azur
SURESNES: Île-de-France
TOURCOING: Hauts-de-France
ROSNY SUR SEINE: 

KeyboardInterrupt: 

In [35]:
df_app['Region'] = df_app['Region'].apply(lambda x: None if isinstance(x, str) and x.startswith("No results found") else x)

In [36]:
df_app['Region'] = df_app['Region'].apply(lambda x: None if isinstance(x, str) and x.startswith("No region found") else x)

In [38]:
df_app['Region'].value_counts()

Region
Île-de-France                 21981
Hauts-de-France               12924
Auvergne-Rhône-Alpes           9025
Provence-Alpes-Côte d'Azur     6568
Occitanie                      6255
Normandie                      5418
Grand Est                      5029
Centre-Val de Loire            3510
Nouvelle-Aquitaine             3468
Pays de la Loire               3461
Bretagne                       2767
Bourgogne-Franche-Comté        2548
Aquitaine                       731
Languedoc-Roussillon            311
La Réunion                      304
Maroc                           238
Tunisie                         182
Côte d'Ivoire                    75
Senegal                          66
France                           59
Cameroon                         56
Martinique                       40
Algerie                          32
Guadeloupe                       24
Guyane                           10
Mayotte                           2
Corse                             2
Haute-Normandie      

In [45]:
# If Mobilite is missing, then we can fill it with the region from the "Region" feature. 
for i in range(len(df_app)):
    if pd.isnull(df_app['Mobilité'][i]) and pd.notnull(df_app['Region'][i]):
        df_app.loc[i, 'Mobilité'] = df_app['Region'][i]





In [28]:
df_app.shape

(85087, 17)

In [30]:
df_app.isnull().sum()

Unnamed: 0                                        0
Id_application                                    0
Date                                              0
Id_candidate                                      0
Id_vacancy                                        0
Civilité                                      10107
Ville                                             0
status                                            0
Motif de refus                                84087
Métier                                          439
Tranche d'age                                 84221
Type de contrat                               85087
Prétentions salariales mensuelles fixe net    60252
Permis de conduire                            56033
Mobilité                                          0
Code postal                                    3984
Region                                            0
dtype: int64

In [ ]:
#df_app.to_csv('applications_updated.csv')

### Merging the three datasets and creating a json dataset for sentence transformers

In [31]:
merged_df = df_app.merge(df_candidate, on='Id_candidate', how='left')

merged_df = merged_df.merge(df_vac_textual, on='Id_vacancy', how='left')


In [32]:
merged_df.shape

(85087, 28)

In [33]:
merged_df.isnull().sum()

Unnamed: 0                                        0
Id_application                                    0
Date                                              0
Id_candidate                                      0
Id_vacancy                                        0
Civilité                                      10107
Ville                                             0
status                                            0
Motif de refus                                84087
Métier                                          439
Tranche d'age                                 84221
Type de contrat                               85087
Prétentions salariales mensuelles fixe net    60252
Permis de conduire                            56033
Mobilité                                          0
Code postal                                    3984
Region                                            0
name                                          29128
fileName                                      29128
nationality 

In [34]:
merged_df.dropna(subset=['text', 'vacancy_text', 'experiences'], inplace=True)
merged_df.shape

(54440, 28)

In [35]:
# We already have a column for vacancy text. Let's create a column for candidate text combining the text, experiences, and Mobilite columns 

merged_df['candidate_text'] = (
	merged_df['text'].astype(str)
	+ "\nEXPERIENCES:\n"
	+ merged_df['experiences'].astype(str)
	+ "\nMOBILITE:\n"
	+ merged_df['Mobilité'].astype(str)
)

In [36]:
# Let us see what the raw candidate text looks like:

merged_df['candidate_text'][16]

"DELEPLANQUE Aurele COMPETENCES Capacité d’analyse, d’écoute, de synthèse, autonomie, gestion des priorités Goût du challenge Maitrise du pack office CONTACT 18 rue des ormeaux 59175 Vendeville 06 58 43 34 20 mailto:adeleplanque@live.fr adeleplanque@live.fr SITUATION Née le 03/11/1984 Permis B et véhicule POINTS FORTS Réactive Dynamique Rigoureuse Capacité d’adaptation Gestion du stress INTERETS Voyages Lecture Cuisine ASSISTANTE ADV PARCOURS PROFESSIONNEL SECURIRACKFRANCE–ASSISTANTEADMINISTRATIVEETCO MMERCIALE Depuis septembre 2020 Création de contrats, de devis Gestion administrative du personnel Accueil physique et téléphonique-courriers Préparation à la comptabilité Facturation et relance (clients-fournisseurs) Préparation de salons professionnels Organisation du planning des opérations Réservations des logements lors des chantiers Gestion du stock Gestion des expéditions Organisation et suivi des transports LOXAMA CCE SS-FRETIN–RESPONSABLEDELOCATION/BTOB 2018 – 2020 Contrats clien

In [37]:
# Cleaning the candidate text by removing unncessary information like email Id and phone numbers

import re

def clean_text(text):
    if pd.isnull(text):
        return text
    
    # Remove email addresses
    text = re.sub(r'\b[\w.-]+?@\w+?\.\w+?\b', '', text)
    
    # Remove phone numbers (various formats)
    text = re.sub(r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,5}[-.\s]?\d{3,5}\b', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply the function to the column
merged_df['candidate_text'] = merged_df['candidate_text'].apply(clean_text)

In [38]:
merged_df['candidate_text'][16]

"DELEPLANQUE Aurele COMPETENCES Capacité d’analyse, d’écoute, de synthèse, autonomie, gestion des priorités Goût du challenge Maitrise du pack office CONTACT 18 rue des ormeaux 59175 Vendeville 06 58 43 34 20 mailto: SITUATION Née le 03/11/1984 Permis B et véhicule POINTS FORTS Réactive Dynamique Rigoureuse Capacité d’adaptation Gestion du stress INTERETS Voyages Lecture Cuisine ASSISTANTE ADV PARCOURS PROFESSIONNEL SECURIRACKFRANCE–ASSISTANTEADMINISTRATIVEETCO MMERCIALE Depuis septembre 2020 Création de contrats, de devis Gestion administrative du personnel Accueil physique et téléphonique-courriers Préparation à la comptabilité Facturation et relance (clients-fournisseurs) Préparation de salons professionnels Organisation du planning des opérations Réservations des logements lors des chantiers Gestion du stock Gestion des expéditions Organisation et suivi des transports LOXAMA CCE SS-FRETIN–RESPONSABLEDELOCATION/BTOB 2018 – 2020 Contrats clients : offres de prix, édition des factures

### Assigning labels

In [41]:
status_1 = {'Embauche'}

status_2 = {'NAE', 'NAE +5jrs', 'NAE Vivier'}

status_3 = {'NET', 'NET+ 3jrs', 'NET Savoir Etre', 'NET Rému', 'NET Cond. Travail', 'NET Localisation',
                'NET Savoir Faire'}

status_4 = {'Négatif', 'Négatif + 2jrs', 'Négatif + 5jrs', 'Neg sans msg'}

other = {'Vivier', 'Vivier + 2jrs', 'No news', 'Poste pourvu'}

for i in merged_df.index:
    if merged_df.loc[i, 'status'] in status_1:
        merged_df.loc[i, 'label'] = 1
    elif merged_df.loc[i, 'status'] in other:
        merged_df.loc[i, 'label'] = None
    else:
        merged_df.loc[i, 'label'] = 0






In [42]:
merged_df.shape

(54440, 30)

In [ ]:
merged_df['label'].value_counts()

label
0.0    34720
1.0      417
Name: count, dtype: int64

In [44]:
merged_df['label'].isnull().sum()

19303

### Converting to json for the sentence transformers

In [45]:
# We need the following columns for this: 'vacancy_text', 'candidate_text', 'Civilite', 'label'. 
# First, recheck for missing values in any of these columns

merged_df.isnull().sum()

Unnamed: 0                                        0
Id_application                                    0
Date                                              0
Id_candidate                                      0
Id_vacancy                                        0
Civilité                                       6367
Ville                                             0
status                                            0
Motif de refus                                53454
Métier                                           17
Tranche d'age                                 54035
Type de contrat                               54440
Prétentions salariales mensuelles fixe net    39649
Permis de conduire                            38450
Mobilité                                          0
Code postal                                    2452
Region                                            0
name                                              0
fileName                                          0
nationality 

In [90]:
# Still some missing values in Civilite, delete them. 

merged_df = merged_df.dropna(subset=['Civilité'])

In [91]:
merged_df.shape

(46279, 29)

In [ ]:
merged_df['Civilité'].value_counts()

# Distribution of gender 

Civilité
Monsieur    34416
Madame      11863
Name: count, dtype: int64

In [93]:
# Distribution of labels

merged_df['label'].value_counts()

label
0    28576
1    17703
Name: count, dtype: int64

In [94]:
# Label versus gender

pd.crosstab(merged_df['label'], merged_df['Civilité'])

Civilité,Madame,Monsieur
label,,
0,7734,20842
1,4129,13574


In [95]:
# Mapping gender to a number 

def map_gender_to_number(gender):
    if gender == 'Madame':
        return 1
    elif gender == 'Monsieur':
        return 0

merged_df['gender_number'] = merged_df['Civilité'].apply(map_gender_to_number)

In [ ]:
#merged_df.to_csv('new_everything_dataset.csv')

In [98]:
df_renamed = merged_df[['candidate_text', 'vacancy_text', 'gender_number', 'label']].rename(columns={
    'candidate_text': 'sentence1',
    'vacancy_text': 'sentence2'
})

df_renamed.head()

,sentence1,sentence2,gender_number,label
16,DELEPLANQUE Aurele COMPETENCES Capacité d’anal...,Titre: Chargé de relation Clientèle (H/F) - Ma...,1,1
50,ALBICE Yohann 2 rue d'Alembert 93000 BOBIGNY 2...,Titre: Mécanicien spécialisé (H/F) - Gennevill...,0,1
55,Jeremy Philétas Electromécanicien Ozoir-la-Fer...,Titre: Electro-mécanicien (H/F) - Croissy Beau...,0,1
234,CONDUCTEUR DÉMONSTRATEUR PL CONTACT 553 CHEMIN...,Titre: Conducteur VL (H/F) - Lyon Sud\nFonctio...,0,1
299,Fabrice Tallon Neuilly-sous-Clermont (60) 6060...,Titre: Mécanicien Spécialisé (H/F) - Saint Max...,0,1


In [99]:
df_renamed.shape

(46279, 4)

In [100]:
# Convert to list of dictionaries
json_data = df_renamed.to_dict(orient='records')

# Save to JSON file
with open('data_made_with_raw_cv_text.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

## Part 2: Make updates to the CV text with language models

In [2]:
df = pd.read_csv('new_everything_dataset.csv')
df.shape

C:\Users\Sinjini\AppData\Local\Temp\ipykernel_11716\1639956706.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('new_everything_dataset.csv')


(54440, 31)

In [3]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Id_application', 'Date', 'Id_candidate',
       'Id_vacancy', 'Civilité', 'Ville', 'status', 'Motif de refus', 'Métier',
       'Tranche d'age', 'Type de contrat',
       'Prétentions salariales mensuelles fixe net', 'Permis de conduire',
       'Mobilité', 'Code postal', 'Region', 'name', 'fileName', 'nationality',
       'permis', 'address', 'experiences', 'text', 'gender', 'postcode',
       'jobTitle', 'vacancy_text', 'candidate_text', 'label'],
      dtype='object')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

template = """
Extrayez les champs structurés suivants du profil du candidat :

- Intitulé du poste actuel
- Intitulés des postes précédents
- Formation
- Compétences
- Secteurs d'activité des entreprises précédentes

Soyez concis et concentrez-vous uniquement sur les mots-clés. Voice le profil: 


{messy_text}
"""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="phi4-mini", temperature=0)
chain = prompt | model

start = time.time()
updates = 0

def process_row(i):
    try:
        messy_text = str(df.loc[i, 'candidate_text'])
        mobility = str(df.loc[i, 'Mobilité'])
        messy_text += f"\nMobilité/Lieu de travail souhaité: {mobility}"

        result = chain.invoke({"messy_text": messy_text})
        print(f"Row {i} update:\n{result}\n{'-'*40}")
        return i, result
    except Exception as e:
        print(f"Error at row {i}: {e}")
    return i, None

# Use multithreading
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_row, i) for i in range(len(df))]
    
    for future in as_completed(futures):
        i, result = future.result()
        if result:
            df.loc[i, 'candidate_text_llm'] = result
            updates += 1
            print(f"Updated row {i}. Total updates so far: {updates}")

#df.to_csv("new_everything_llm.csv", index=False)
print(f"Finished updating {updates} rows in {time.time() - start:.2f} seconds.")


In [2]:
df1 = pd.read_csv('new_everything_llm.csv')

C:\Users\Sinjini\AppData\Local\Temp\ipykernel_22624\950053154.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('new_everything_llm.csv')


In [4]:
df1.shape

(43768, 33)

In [3]:
df1['Civilité'].value_counts()

Civilité
Monsieur    32523
Madame      11245
Name: count, dtype: int64

In [6]:
df1['label'].value_counts()

label
0    27044
1    16724
Name: count, dtype: int64

In [7]:
pd.crosstab(df1['label'], df1['Civilité'])

Civilité,Madame,Monsieur
label,,
0,7329,19715
1,3916,12808


In [36]:
df_renamed = df1[['structured_profile', 'vacancy_text', 'gender_number', 'label']].rename(columns={
    'structured_profile': 'sentence1',
    'vacancy_text': 'sentence2'
})

df_renamed.head()

,sentence1,sentence2,gender_number,label
0,:\n\n- **Intitulé du poste actuel** : Assistan...,Titre: Chargé de relation Clientèle (H/F) - Ma...,1,1
1,:\n\n- **Intitulé du poste actuel** : Ouvrier\...,Titre: Mécanicien spécialisé (H/F) - Gennevill...,0,1
2,**Current Title**: septembre\n**Previous Roles...,Titre: Electro-mécanicien (H/F) - Croissy Beau...,0,1
3,:\n\n- **Intitulé du poste actuel** : Conducte...,Titre: Conducteur VL (H/F) - Lyon Sud\nFonctio...,0,1
4,**Current Title**: technicien de maintenance\n...,Titre: Mécanicien Spécialisé (H/F) - Saint Max...,0,1


In [37]:
df_renamed['sentence1'] = df_renamed['sentence1'].str.replace(':\n\n', '')
df_renamed['sentence1'] = df_renamed['sentence1'].str.replace('-**', '')
df_renamed['sentence1'] = df_renamed['sentence1'].str.replace('**', '')
df_renamed['sentence1'] = df_renamed['sentence1'].str.replace('-', '')

In [39]:
# Convert to list of dictionaries
json_data = df_renamed.to_dict(orient='records')

# Save to JSON file
with open('data_with_llm_cv_text.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

## Part 3: Creating sentence pairs with ESCO titles
(This dataset was not used for evaluation or training. A better dataset with ESO titles was created later). 

In [ ]:
df = pd.read_csv("new_everything_dataset.csv")

In [ ]:
df['experiences'][0]

In [ ]:
import ast

def extract_non_blank_esco_titles_ast(experiences_str):
    """
    Extracts non-blank 'escoTitle' values from a string representation of a list of dictionaries
    using ast.literal_eval to handle potentially non-strict JSON.

    Args:
        experiences_str: A string representing a list of dictionaries, where each
                         dictionary contains job experience information, including
                         'escoTitle'.

    Returns:
        A list of non-blank 'escoTitle' strings found in the input.
    """
    try:
        experiences_list = ast.literal_eval(experiences_str)
        esco_titles = [exp['escoTitle'] for exp in experiences_list if isinstance(exp, dict) and exp.get('escoTitle')]
        return esco_titles
    except (SyntaxError, ValueError):
        print(f"Error decoding string with ast.literal_eval: {experiences_str}")
        return []

# Apply the corrected function to the 'experiences' column
df['esco_titles'] = df['experiences'].apply(extract_non_blank_esco_titles_ast)

# You can still explode the DataFrame if needed
df_exploded = df.explode('esco_titles')
print(df_exploded[['jobTitle', 'esco_titles']].head())

In [ ]:
# Calculate the number of rows where 'esco_titles' is an empty list
num_blank_esco_titles = df['esco_titles'].apply(lambda x: len(x) == 0).sum()

# Print the result
print(f"Number of blank esco_titles: {num_blank_esco_titles}")

In [ ]:
count = 0
for i in range(len(df['esco_titles'])):
    if len(df['esco_titles'][i]) == 0 or not isinstance(df['esco_titles'][i], list) or df['esco_titles'][i] == []:
        count += 1
        #print(i)

print(count)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

esco_df = pd.read_csv("occupations_fr.csv")

model = SentenceTransformer('distiluse-base-multilingual-cased')  # Supports French

# Encode ESCO titles
esco_embeddings = model.encode(esco_df['description'].tolist(), convert_to_tensor=True)

def get_best_esco_title(text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    similarities = cosine_similarity([text_embedding], esco_embeddings)[0]
    best_match_idx = similarities.argmax()
    return str(esco_df.iloc[best_match_idx]['preferredLabel']) + str(esco_df.iloc[best_match_idx]['altLabels'])

# Apply to your dataframe
#df['esco_title_vacancy'] = df['vacancy_text'].apply(get_best_esco_title)


In [ ]:
for element in df['vacancy_text'][:1]:
    print(get_best_esco_title(element))

In [ ]:
get_best_esco_title(df['vacancy_text'][0])

In [ ]:
# Apply to your dataframe
df['esco_title_vacancy'] = df['vacancy_text'].apply(get_best_esco_title)

In [ ]:
df.to_csv('new_everything_esco.csv')

In [ ]:
df = pd.read_csv("new_everything_esco.csv")

In [ ]:
df.rename(columns={'esco_titles': 'esco_titles_candidate'}, inplace=True)

In [ ]:
#How many rows have empty esco_titles_candidate?
df['esco_titles_candidate'][df['esco_titles_candidate'].isnull() | (df['esco_titles_candidate'] == '[]')].shape[0]

In [ ]:
# How many rows have empty esco_titles_vacancy?
df['esco_title_vacancy'][df['esco_title_vacancy'].isnull() | (df['esco_title_vacancy'] == '[]') | (df['esco_title_vacancy'] == '')].shape[0]


In [ ]:
# Delete the rows with empty esco_titles_candidate

df = df[~(df['esco_titles_candidate'].isnull() | (df['esco_titles_candidate'] == '[]'))]

In [ ]:
df['esco_titles_candidate'] = df['esco_titles_candidate'].str.replace('[', '')
df['esco_titles_candidate'] = df['esco_titles_candidate'].str.replace(']', '')
df['esco_titles_candidate'] = df['esco_titles_candidate'].str.replace("'", '')

In [ ]:
df_renamed = df[['esco_titles_candidate', 'esco_title_vacancy', 'gender_number', 'label']].rename(columns={
    'esco_titles_candidate': 'sentence1',
    'esco_title_vacancy': 'sentence2'
})

df_renamed.head()

In [ ]:
# Convert to list of dictionaries
json_data = df_renamed.to_dict(orient='records')

# Save to JSON file
with open('data_with_esco_titles.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)